
# AI Detective Lab 02 – Training the Tools Before the Next Breakthrough

In the last homework, you made some progress on the case. But the chief is getting impatient.

> *"I don't want vibes, I want **models**,"* she says.  
> *"Before you tackle the full estate data again, go back to the lab and get your tools in order."*

This lab is all about **syntax and mechanics**. The ideas of decision trees, probabilities, and the Viterbi algorithm will be covered in lecture. Here, you will:

- Practice turning narrative **statements** into numeric feature vectors.
- Train and inspect a simple **decision tree classifier** in scikit-learn.
- See why long chains of probabilities should be computed in **log space**.
- Implement and run a small **Viterbi algorithm** on a tiny map of rooms.
- Combine scores from different models into a single probability distribution.

This lab is meant to be *gentle*. It is **not** trying to trick you. If you can get through this notebook, you will have most of the **syntax** you need for Homework 02.



## 0. Lab Setup

Run the cell below to import the libraries we will use.

If you see any errors, let your instructor or TA know.


In [ ]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier, export_text

print("NumPy version:", np.__version__)
print("pandas version:", pd.__version__)


## 1. From Narrative Statements to Features

You return to the evidence board. There are stacks of **interview notes**—descriptions of where different people claimed to be, and what they were doing.

Unfortunately, computers don't understand sentences like:

> "I was reading in the Study, and then I went to make tea in the Kitchen."

We need to turn these into **numeric clues**: 0/1 features that say things like:

- `in_kitchen = 1` if the Kitchen is mentioned, else 0.
- `in_study = 1` if the Study is mentioned, else 0.
- `mentions_knife = 1` if a knife is mentioned, else 0.

Below is a tiny dataset of *toy* statements to get you started.


In [ ]:
toy_statements = [
    {
        "suspect": "Alex",
        "activities": ["Drinking tea", "Reading in the Study"],
        "weapon_mention": "Candlestick"
    },
    {
        "suspect": "Blair",
        "activities": ["Cooking in the Kitchen"],
        "weapon_mention": "Knife"
    },
    {
        "suspect": "Casey",
        "activities": ["Walking through the Hall", "Checking the Study"],
        "weapon_mention": "None"
    },
]

toy_statements


### 1.1 Implement `extract_features`

Fill in the function below so that it converts **one** statement dictionary into a dictionary of numeric features.

We will start with just a few features:

- `in_kitchen`: 1 if any activity mentions `"Kitchen"`, else 0.
- `in_study`: 1 if any activity mentions `"Study"`, else 0.
- `in_hall`: 1 if any activity mentions `"Hall"`, else 0.
- `mentions_knife`: 1 if the weapon mention is exactly `"Knife"`, else 0.
- `mentions_candlestick`: 1 if the weapon mention is exactly `"Candlestick"`, else 0.

The starter code includes an example of using `any("Kitchen" in act for act in activities)`.


In [ ]:
def extract_features(statement):
    """
    Convert one statement dict into a flat dict of numeric features (0/1).
    statement has keys: "suspect", "activities", "weapon_mention".
    """
    features = {}
    
    activities = statement["activities"]
    weapon = statement["weapon_mention"]
    
    # TODO: Set features based on activities and weapon.
    # HINT: use any("Kitchen" in act for act in activities) to check if
    # any activity string contains the word "Kitchen". Then convert True/False
    # to 1/0 using int(...).
    
    features["in_kitchen"] = int(any("Kitchen" in act for act in activities))
    features["in_study"] = int(any("Study" in act for act in activities))
    features["in_hall"] = int(any("Hall" in act for act in activities))
    
    features["mentions_knife"] = int(weapon == "Knife")
    features["mentions_candlestick"] = int(weapon == "Candlestick")
    
    return features

# Quick test: run extract_features on each toy statement.
for s in toy_statements:
    print(s["suspect"], "->", extract_features(s))


### 1.2 Turn Feature Dicts into a DataFrame

Machine learning libraries like scikit-learn expect the input features to be in a **matrix** form: rows are examples, columns are features.

The code below:

- Applies your `extract_features` function to each statement.
- Builds a pandas `DataFrame` from the resulting list of feature dicts.


In [ ]:
feature_dicts = [extract_features(s) for s in toy_statements]
df_features = pd.DataFrame(feature_dicts)
df_features


Use the table above to answer these questions (you do **not** need to write code for this):

1. Which suspect has `in_kitchen = 1`?
2. Which suspect has both `in_study = 1` and `mentions_candlestick = 1`?
3. How might we extend this to include more detailed features (e.g., time of day, number of rooms mentioned, etc.)?



## 2. Your First Decision Tree: Is the Alibi Plausible?

Now that we have numeric clues, we can train a **Decision Tree** to classify whether an alibi seems **plausible** or **suspicious**.

For this toy example, we'll just assign labels by hand, pretending these came from an experienced detective.


In [ ]:
# Toy labels that we imagine an expert detective provided.
# (In the real homework, you'll get labels from the dataset.)
labels = [
    "Suspicious",   # Alex: Study + Candlestick feels a bit on-the-nose...
    "Plausible",    # Blair: just cooking in the Kitchen
    "Suspicious",   # Casey: wandering the Hall and Study at odd times
]

y = np.array(labels)
X = df_features.to_numpy()

print("X shape:", X.shape)  # (num_examples, num_features)
print("y shape:", y.shape)


### 2.1 Train a Decision Tree Classifier

We will use scikit-learn's `DecisionTreeClassifier`:

- `criterion="entropy"`: splits based on information gain (as in lecture).
- `max_depth=3`: prevents the tree from growing too deep.
- `random_state=42`: makes the results reproducible.

Run the cell and make sure it completes without error.


In [ ]:
clf = DecisionTreeClassifier(
    criterion="entropy",
    max_depth=3,
    random_state=42
)
clf.fit(X, y)

print("Tree trained!")


### 2.2 Predict for a New Statement

Let's imagine a new suspect, **Dana**, who claims:

> "I was reading in the Study, and then I went for a walk in the Hall."

We'll build a statement dictionary, convert it to features using `extract_features`, and then call:

- `clf.predict(...)` for the predicted label.
- `clf.predict_proba(...)` for the probabilities of each class.


In [ ]:
new_statement = {
    "suspect": "Dana",
    "activities": ["Reading in the Study", "Walking through the Hall"],
    "weapon_mention": "None"
}

new_features = extract_features(new_statement)
new_df = pd.DataFrame([new_features])

print("Features for Dana:")
display(new_df)

print("Prediction:", clf.predict(new_df)[0])
print("Probabilities (class order):", clf.classes_)
print(clf.predict_proba(new_df)[0])


### 2.3 Inspect the Learned Tree

The code below prints a **text version** of the learned decision tree. Compare it to the feature table from Section 1.

- Which feature does the tree split on first?
- Does the tree's decision process make intuitive sense given how we labeled the examples?


In [ ]:
tree_text = export_text(clf, feature_names=list(df_features.columns))
print(tree_text)


## 3. Probabilities in Code & Why We Use Log Space

In the estate case, you will often need to multiply **many** probabilities together:

- Transition probabilities between rooms.
- Emission probabilities of observations.
- Prior probabilities for suspects or aliases.

If you multiply enough numbers between 0 and 1, you quickly get **very tiny** values that computers round down to 0. This is called **underflow**.

To avoid this, we work in **log space**:

- Instead of multiplying probabilities, we add their logs:
  \[
  \log(a \cdot b \cdot c) = \log a + \log b + \log c
  \]



### 3.1 Simple Example: Direct Product vs Log Space


In [ ]:
probs = np.array([0.8, 0.7, 0.9, 0.6, 0.5])

product = np.prod(probs)
print("Direct product:", product)

log_sum = np.sum(np.log(probs))
back_to_prob = np.exp(log_sum)
print("Log-sum-exp result:", back_to_prob)


### 3.2 Longer Chains of Probabilities

What happens if we multiply **50** copies of 0.9? Or 100? Try running the code below and see when the direct product gets extremely small, while the log-space version still works fine.


In [ ]:
for length in [10, 20, 50, 100]:
    long_probs = np.full(length, 0.9)
    product_long = np.prod(long_probs)
    log_sum_long = np.sum(np.log(long_probs))
    back_to_prob_long = np.exp(log_sum_long)
    
    print(f"Length = {length}")
    print("  Direct product:", product_long)
    print("  Log-space result:", back_to_prob_long)
    print()


**Question (no code required):**

In the Viterbi algorithm, we repeatedly multiply many probabilities together along a path. Why is it safer (numerically) to do this in log space instead of directly multiplying the probabilities?



## 4. A Tiny Hidden Markov Model & Viterbi on a Mini Map

The estate is complicated, but let's start with a **tiny** map of rooms:

- `Study`
- `Kitchen`
- `Hall`

We will define:

- An initial distribution over where the suspect might start.
- A transition matrix telling us how likely they are to move between rooms.
- An emission matrix telling us how consistent each room is with a particular observation (e.g., where a witness claims they were).

Then we will implement a small version of the **Viterbi algorithm** to find the most likely path.


In [ ]:
rooms = ["Study", "Kitchen", "Hall"]
room_to_id = {r: i for i, r in enumerate(rooms)}
n_states = len(rooms)

# Transition probabilities: rows = current room, columns = next room
transition = np.array([
    [0.7, 0.2, 0.1],  # from Study
    [0.3, 0.5, 0.2],  # from Kitchen
    [0.2, 0.3, 0.5],  # from Hall
])

# Initial distribution over rooms
initial = np.array([1/3, 1/3, 1/3])

# Emissions: for each timestep, a distribution over rooms
# Imagine three "observations" over time (t1, t2, t3)
timestamps = ["t1", "t2", "t3"]
emissions = np.array([
    [0.9, 0.05, 0.05],  # t1: strong evidence for Study
    [0.1, 0.8, 0.1],    # t2: strong evidence for Kitchen
    [0.2, 0.2, 0.6],    # t3: strong evidence for Hall
])

transition, initial, emissions


### 4.1 Implement `viterbi_simple`

Fill in the missing pieces of the `viterbi_simple` function below.

We will:

1. Convert `initial`, `transition`, and `emissions` to **log space**.
2. Use a dynamic programming table `V[t, s]` for the best log-probability of ending in state `s` at time `t`.
3. Keep a `backptr[t, s]` that remembers which previous state led to the best score.
4. After filling in the table, backtrack from the best final state to recover the full path.

**Important:** We will work in log space the whole time. That means:

- Initialization: `V[0] = log(initial) + log(emissions[0])`
- Transition step: for each `t` and `s`, compute
  \[
  \max_j V[t-1, j] + \log(transition_{j,s})
  \]
  and then add `log(emissions[t, s])`.


In [ ]:
def viterbi_simple(initial, transition, emissions, rooms):
    T = emissions.shape[0]   # number of timesteps
    N = emissions.shape[1]   # number of states
    
    # Convert everything to log space
    log_init = np.log(initial)
    log_trans = np.log(transition)
    log_emit = np.log(emissions)
    
    # DP table for log probabilities
    V = np.zeros((T, N))
    # Backpointers
    backptr = np.zeros((T, N), dtype=int)
    
    # TODO: Initialize V[0] using log_init and log_emit[0]
    V[0] = log_init + log_emit[0]
    
    # TODO: Fill in the DP table
    for t in range(1, T):
        for s in range(N):
            # scores if we came from each previous state j
            scores = V[t-1] + log_trans[:, s]
            best_prev = np.argmax(scores)
            V[t, s] = scores[best_prev] + log_emit[t, s]
            backptr[t, s] = best_prev
    
    # TODO: Backtrack to find the best path
    best_last_state = np.argmax(V[T-1])
    best_path_ids = [best_last_state]
    
    for t in range(T-1, 0, -1):
        best_last_state = backptr[t, best_last_state]
        best_path_ids.append(best_last_state)
    
    best_path_ids.reverse()
    best_path_rooms = [rooms[i] for i in best_path_ids]
    best_log_prob = V[T-1, np.argmax(V[T-1])]
    
    return best_path_rooms, np.exp(best_log_prob)

# Run Viterbi on the tiny example
path, prob = viterbi_simple(initial, transition, emissions, rooms)
print("Most likely path:", path)
print("Path probability:", prob)


**Reflection (no code required):**

1. What does `V[t, s]` represent in plain English?
2. What does `backptr[t, s]` remember?
3. How does this implementation relate to the Viterbi diagrams we draw in lecture?



## 5. Combining Evidence: Tree Score + Path Likelihood

In the full homework, you will have multiple sources of evidence:

- A model that analyzes **statements** (e.g., decision tree on features).
- A model that analyzes **movement** (e.g., HMM/Viterbi on room paths).

A simple way to combine different scores is to multiply them (assuming, roughly, that they provide independent evidence) and then **normalize** so the scores sum to 1.


In [ ]:
suspects = ["Alex", "Blair", "Casey"]

# Example: probability of being alias "Mr. Green" from the decision tree model
tree_scores = np.array([0.6, 0.2, 0.4])

# Example: probability that their path matches a "suspicious movement" pattern
path_scores = np.array([0.3, 0.7, 0.5])

print("Tree scores:", tree_scores)
print("Path scores:", path_scores)


### 5.1 Combine and Normalize

Complete the code below to:

1. Compute the **combined score** as the elementwise product of `tree_scores` and `path_scores`.
2. Normalize the combined scores so they sum to 1.

Then interpret the results: who looks most suspicious according to this simple combination rule?


In [ ]:
# TODO: combine scores by multiplying them
combined = tree_scores * path_scores

# TODO: normalize so the combined scores sum to 1
combined = combined / combined.sum()

for name, score in zip(suspects, combined):
    print(f"{name}: {score:.3f}")


**Question (no code required):**

If being "Mr. Green" means **both** sounding suspicious in statements and having a weird path through the estate, why might multiplying these scores be a reasonable (though simple) way to combine them?



## 6. Lab Checklist & Bridge to Homework 02

Before you move on to Homework 02, make sure you feel comfortable with:

- ✅ Writing functions that:
  - Take a statement dict and return a numeric feature dict.
  - Loop through many statements and build a feature matrix (`DataFrame`).
- ✅ Fitting a `DecisionTreeClassifier` with:
  - `clf = DecisionTreeClassifier(criterion="entropy", max_depth=3, random_state=42)`
  - `clf.fit(X, y)` and `clf.predict_proba(X_new)`
- ✅ Working with probabilities and understanding:
  - Why multiplying many probabilities can underflow.
  - How log space fixes that (`np.log`, `np.exp`).
- ✅ Implementing Viterbi on a small HMM:
  - Setting up `initial`, `transition`, `emissions`.
  - Filling in `V` and `backptr` and backtracking to get the best path.
- ✅ Normalizing a vector of scores so it sums to 1.

When you get stuck on syntax in the homework, come back to this lab and use it as a reference.

Good luck, Detective.
